# Ansoegning

### Importering af biblioteker

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


#### Øge pandas maksimale visning af rækker

In [2]:
pd.set_option('display.max_rows', None)

### Løsning af CSV fil

In [3]:
# Den statiske filsti
filepath = r"C:\Users\steff\OneDrive\Dokumenter\Ansøgninger 2024\Sikkerhedsstyrelsen\PythonUdvikler\husrapporter-2021-01.csv"

# Avender pandas til at læse csv filen
data = pd.read_csv(filepath)


### Overblik over data

In [4]:
# Overblik over headers og de første fem datapoints af datasættet
data.head()

,rapport_id,inspektoer_id,ejendoms_id,oprettelsesdato,besoegsdato,gyldighedsdato
0,47914,1018,47913,2021-01-04 10:02:30,2021-01-11 08:00:00,2021-01-11 10:38:45
1,48969,619,48968,2021-01-05 13:22:08,2021-01-05 11:00:00,2021-01-05 13:32:03
2,50178,368,50177,2021-01-07 10:44:30,2021-01-08 11:43:00,2021-01-08 14:34:58
3,51212,608,50177,2021-01-10 21:43:22,2021-01-08 07:00:00,2021-01-10 22:42:58
4,49567,638,49566,2021-01-06 11:42:07,2021-01-06 09:30:00,2021-01-08 10:22:50


### Her kan vi se at vi har følgende data:
- rapport_id:           Raportens ID
- inspektoer_id:        ID på den inspektør som har udført opgaven
- ejendoms_id:          ID på den ejensom der er udført kontrol på
- oprettelsesdato:      Dato for oprettelse
- besoegsdato:          Dato for besøg af inspektoer
- gyldighedsdato:       Dato for gyldighed

# Opgaver


## Vi vil gerne vide om der er signifikant forskel på tiderne hos de forskellige inspektører og fagområder.

### Kan vi klassificere inspektørerne efter de to fagområder?

Ved første gennemgang af data, kan vi se at flere ejendomme har et eller flere ID tilknyttet forskellige inspektører. De inspektører som er de eneste tilknyttet en ejendoms ID vil således være de som kan udarbejde både elinstallations- og tilstandsrapport.

#### List med antal rapporter hver ejendom har

In [5]:
# Identificer ejendomme med præcis én rapport og mere end én rapport
ejendomme_antal_rapporter = data['ejendoms_id'].value_counts()
ejendomme_antal_rapporter.value_counts()

2    5307
1    2416
3     110
4      10
Name: ejendoms_id, dtype: int64

Her har vi en fordeling af hvor mange rapporter hver ejendom har liggende, fordelt fra én op til fire rapporter.

#### Antal ejendomme med én eller flere rapporter

In [6]:
# Ejendomme med præcis én rapport
en_rapport_ejendomme = ejendomme_antal_rapporter[ejendomme_antal_rapporter == 1].index.tolist()

# Ejendomme med mere end én rapport
flere_rapporter_ejendomme = ejendomme_antal_rapporter[ejendomme_antal_rapporter > 1].index.tolist()

# Antal ejendomme med hhv. én og flere rapporter
antal_ejendomme_en_rapport = len(en_rapport_ejendomme)
antal_ejendomme_flere_rapporter = len(flere_rapporter_ejendomme)

(antal_ejendomme_en_rapport, antal_ejendomme_flere_rapporter)

(2416, 5427)

Her kan vi se at 5427 ejendomme har mere end én rapport mens 2416 kun har én rapport liggende.

#### Datasæt for ejendomme med hhv. én eller flere rapporter

In [7]:
# Data for ejendomme med præcis én rapport
data_en_rapport = data[data['ejendoms_id'].isin(en_rapport_ejendomme)]

# Data for ejendomme med mere end én rapport
data_flere_rapporter = data[data['ejendoms_id'].isin(flere_rapporter_ejendomme)]                                                   

#### Inspektører der kan udarbejde både tilstands- og elinstallationsrapport

In [8]:
inspektør_to_rapporter = data_en_rapport['inspektoer_id'].unique()
antal_inspektoerer_en_rapport = len(inspektør_to_rapporter)
antal_inspektoerer_en_rapport

469

Dette tal viser hvor mange inspektører der selvstændigt har været ude ved ejendomme og derved de inspektører som man må antage kan udføre både elinstallations- og tilstandsrepporter.

#### Antal inspektører som er specialiceret i enten elinstallationsrapporter eller tilstandrapporter.

In [9]:
# Beregn antallet af inspektører som ikke kan udarbejde både el- og tilstandsrapporter
inspektoerer_en_rapport = data_flere_rapporter['inspektoer_id'].unique()
unikke_inspektoerer_en_rapport = set(inspektoerer_en_rapport) - set(inspektør_to_rapporter)
antal_unikke_inspektoerer_en_rapport = len(unikke_inspektoerer_en_rapport)
antal_unikke_inspektoerer_en_rapport

121

### Konklusion

Baseret på ovenstående analyse har vi identificereret:

- 2416 ejendomme har én rapport. Her må inspektøren således være i stand til at kunne udføre både elinstallations- og tilstandsrapport. Der er på disse 2416 ejendomme 469 unikke ID'er og derved 469 inspektører som kan varetage begge rapporter.

- 5427 ejemdomme har mere end én rapport, hvilken indikerer at der har været flere inspektører ude for at foretage forskellige rapporttyper. Her er der 556 unikke ID'er som har varetaget disse rapporter.

- Der er 590 unikke inspektører tilknyttet dette datasæt. Hvor 469 inspektører kan varetage både elinstallations- og tilstandsrapporter mens 121 inspektører kan varetage enten den éne eller den anden rapport.

## Vi kan definere færdiggørelsestiden som tiden fra besigtigelse til gyldighed.

### Er der signifikant forskel på færdiggørelsestiden på de to fagområder?

Først skal vi beregne færdiggørelsestiden, altså tiden fra besøget til gyldigheden i totale antal timer. Jeg har valgt at bruge timer og ikke dage, da jeg i datsættet kunne se at flere af besøgene er gyldige den samme dag, dog med variation i timerne. Derfor, for at få det mest relevante billede af en mulig forskel mellem de to fagområder vælger jeg at gå med antal timers forskel.

In [10]:
data['besoegsdato'] = pd.to_datetime(data['besoegsdato'], errors='coerce')
data['gyldighedsdato'] = pd.to_datetime(data['gyldighedsdato'], errors='coerce')

# Beregn færdiggørelsestiden i timer
data['færdiggørelsestid_timer'] = (data['gyldighedsdato'] - data['besoegsdato']).dt.total_seconds() / 3600

# Den gennemsnitlige færdiggørelsestid i timer
data["færdiggørelsestid_timer"].mean()

115.5228491554096

Her kan vi se at der i gennemsnittet bruges 115.5 timer fra besigtigelse til gyldighed af alle rapporter.

#### Dernæst skal vi have fundet den gennemsnitlige færdiggørelsestid grupperne i mellem

In [11]:
# Datasæt for inspektører der kan udfører to rapporter
data_to_rapporter = data[data['inspektoer_id'].isin(inspektør_to_rapporter)]

# Datasæt for inspektører der kan udføre én rapport
data_en_rapport_unik = data[data['inspektoer_id'].isin(unikke_inspektoerer_en_rapport)]

# Den gennemsnitlige færdiggørelsestid for inspektrører der kan udfører to rapporter
average_completion_time_to_rapporter = data_to_rapporter['færdiggørelsestid_timer'].mean()

# Den gennemsnitlige færdiggørelsestid for inspektrører der kan udfører én rapport
average_completion_time_en_rapport = data_en_rapport_unik['færdiggørelsestid_timer'].mean()

(average_completion_time_to_rapporter, average_completion_time_en_rapport)

(119.38515962490322, 36.71567403265885)

Her ser vi, at den gennemsnitlige færdiggørelsestid hos de inspektører som kan udføre begge rapporter ligger på 119 timer, mens det for de inspektører som kun varetager én rapport tager 36 timer.

#### Validering af data

For at validere disse gennemsnitlige færdiggørelsestider vil jeg først lige gennemgå data for at sikre, at disse er korrekt. Og jeg starter med lige at se på færdiggørelses_timer


In [12]:
data['færdiggørelsestid_timer']

0             2.645833
1             2.534167
2             2.866111
3            63.716111
4            48.880556
5            24.125278
6            24.203056
7             2.018889
8            39.660278
9            -0.808889
10            4.802778
11           -0.975556
12           13.017222
13          547.667222
14          644.319167
15          120.084167
16           51.760556
17           15.742222
18           28.603333
19            4.222222
20            5.277222
21            0.196944
22           21.161667
23           79.928333
24           27.413889
25          414.458056
26            0.198889
27            1.695278
28           -5.426667
29           10.955833
30            7.402500
31           23.080278
32           56.463056
33           -1.142778
34           20.160278
35           21.723333
36            2.382500
37           18.891111
38            2.919722
39           21.325000
40            4.460556
41            7.296667
42           28.489722
43         

Vi kan se på dise data at nogle af færdiggørelsestid_timerne er i minus. Det ser altså ud som om at nogle af rapporterne er færdige før at inspektørerne har været ude og besøge ejendommene. Men for at være sikker, så lad vil jeg manuelt kigge på ID 28 som har en forskel på -5.426667.

In [13]:
(data['besoegsdato'][28], data['gyldighedsdato'][28])


(Timestamp('2021-01-07 13:22:00'), Timestamp('2021-01-07 07:56:24'))

Som vi kan se ud fra vores validering af data, stemmer disse overens. Der er altså i datasættet flere steder med en færdiggørelsestid på -. Jeg vil dog mene at dette ikke burde kunne lade sig gøre, da man først kan have en gyldig rapport, når bolige er beset. Derfor vælger jeg at fjerne disse fra datasættet og køre min analyse igen.

In [14]:
# Filtrerer således at færdiggørelsestid_timer som ligger under 0
data_filtered = data[data['færdiggørelsestid_timer'] >= 0]

# Apply denne filtrering på både ejemdoms_id og inspektør_id
data_en_rapport_filtered = data_filtered[data_filtered['ejendoms_id'].isin(en_rapport_ejendomme)]
data_flere_rapporter_filtered = data_filtered[data_filtered['ejendoms_id'].isin(flere_rapporter_ejendomme)]

# Re-identify the groups of inspectors based on the filtered data
data_to_rapporter_filtered = data_filtered[data_filtered['inspektoer_id'].isin(inspektør_to_rapporter)]
data_en_rapport_unik_filtered = data_filtered[data_filtered['inspektoer_id'].isin(unikke_inspektoerer_en_rapport)]

# Beregning af den gennemsnitlige færdiggørelsestid_timer.
average_completion_time_to_rapporter_filtered = data_to_rapporter_filtered['færdiggørelsestid_timer'].mean()
average_completion_time_en_rapport_filtered = data_en_rapport_unik_filtered['færdiggørelsestid_timer'].mean()

(average_completion_time_to_rapporter_filtered, average_completion_time_en_rapport_filtered)

(136.3570012345679, 43.02614021650327)

Som vi kan se ovenfor ændrer den gennemsnitlige færdiggørelsestid sig ift. den forrige beregning. Nu får vi at inspektrører som varetager begge rapporter tager i gennemsnittet 136.5 timer (en forskel på godt 12,5 timer) mens de inspektører som blot kan varetage én tager nu 43 timer (en stigning på over 6 timer.

#### Er det signifikant forskel?

For at beregne om der er signifikant forskel mellem disse to tider, skal vi først finde den analyse metode som passer. Her skal vi først se om færdiggørelsestid_timer data er normaltfordelt.

In [22]:
# Se om data er normalfordelt
normality_to_rapporter_filtered = stats.shapiro(data_to_rapporter_filtered['færdiggørelsestid_timer'].sample(min(5000, len(data_to_rapporter_filtered))))
normality_en_rapport_filtered = stats.shapiro(data_en_rapport_unik_filtered['færdiggørelsestid_timer'].sample(min(5000, len(data_en_rapport_unik_filtered))))

(normality_to_rapporter_filtered, normality_en_rapport_filtered)

(ShapiroResult(statistic=0.004860341548919678, pvalue=0.0),
 ShapiroResult(statistic=0.3102797269821167, pvalue=8.136919792794915e-41))

Som vi kan se er data ikke normalfordelt - dette vises af den signifikante p-værdi som ligger på under 0.000. Derfor skal vi bruge en ikke-parametrisk test for at teste om der er signifikant forskel mellem data. Dette kan gøres via en Mann-Whitney U-test.

In [25]:
# Perform the Mann-Whitney U test on the two groups
mann_whitney_result = stats.mannwhitneyu(data_to_rapporter_filtered['færdiggørelsestid_timer'], 
                                         data_en_rapport_unik_filtered['færdiggørelsestid_timer'], 
                                         alternative='two-sided')

mann_whitney_result


MannwhitneyuResult(statistic=3291481.0, pvalue=0.0028398626086853537)

### Konklusion

Vi kan konkludere at der er en signifikant forskel mellem færdiggørelsestiden mellem de inspektører som kan varetage to rapporter og de som kun varetager en. De som varetager to rapporter bruger i gennemsnittet 93 timer mere end de inspektører som kun varer én rapport. Det giver også mening at tiden vil være forhøjet, da de så skal udføre to rapporter på samme besøg. Dog kan man stille spørgsmålstegn ved, hvorvidt det burde tage så lang tid at gennemføre begge. Her kunne det have været interessant at have data på hvilke rapporter som var elinstallationsrapporter og hvilke der var tilstandsrapporter for at se hvordan forskellen var mellem disse.
Vi kan også konkludere, at der er en signifikant forskel (p = 0.0028) i median færdiggørelsestiden mellem inspektører, der kan håndtere to rapporter, og dem, der kun kan håndtere én rapport, baseret på de filtrerede data.

### Er det inspektøren eller fagområdet som har størst betydning for variationen i færdiggørelsestid?